## Clean Street component of an address

This script uses the python library **usaddress** to parse the street component of an address. The script does some minor preprocessing of the address before passing input to `usaddress`.  We have found these steps to improve geocoding results.

- See: https://parserator.datamade.us/usaddress
   
---

In [ ]:
import os
import pandas as pd
import re
import math
import usaddress

## Big function to pre-clean an address and submit to `usaddress` parser.

In [ ]:
counter = 1  # a global to keep track of number of addresses processed

def get_clean_street(address, debug=0):
    '''
    Function to clean and tag street address component
    Returns a clean street address component if the input type is Street Address
    Else returns minimally cleaned version of the input.
    '''
    
    cleaning_note = ""
    
    global counter
    if debug == 1 :
        print(str(counter)+" input address: " + address)
    
    #add some spaces where they are missing around odd chars
    address =  re.sub('[.\(){}<>\']', ' ', address)
    if debug == 1:
        print("we now have: " + address)
                    
        
    # Dashes 
    # If number-number, remove dash
    # else leave it or replace with a space
    if re.search(r'^\d+-\d+\s+', address):
        # EG '49-866 AVE EL RIO' where 866 is the street name 
        address = re.sub('[.-]', '', address)
        cleaning_note = "Removed dash in num-dash-num|"
        if debug == 1:
            print("we removed dash in num-dash-num and now have: " + address)
     
    # Now if there is a dash still replace with a space
    #'12-23rd st' or '#5-36 Main St'
    if re.search(r'\-', address):
    
        address =  re.sub('[.-]', ' ', address)
        cleaning_note = cleaning_note + "Removed dash in str|"
        
        if debug ==1:
            print("we removed a dash in string and now have: " + address)
        
    # See if there is a forward slash in the address
    # and make sure it has spaces around it
    # if it is not a fraction
    if not re.search(r' \d+/\d+', address):
        if debug > 0:
            print("no fraction so check and fix slashes")
        address = re.sub('[./]', ' ', address)
    
    #strip leading hastags
    address = address.lstrip("#")
    
    # strip leading spaces
    address = address.lstrip()
    
    # replace multiple spaces with one space
    address = re.sub('\s+',' ', address ).strip()
    
    if debug == 1:
        print("Pre-cleaned address: ", address)
    
    ############################################################
    # Tagging address with usaddress
    ############################################################
    
    try:
        tagged_address, address_type = usaddress.tag(address)
   
        if debug == 2 :
            print(tagged_address)

        #addr_type = tagged_addr[1]
        cleaning_note = cleaning_note + "|usaddressType=" + address_type
            
        if debug == 1:
            print("Type of tagged addresss: " + address_type)
            
        if (address_type == 'Street Address'):
            y = tagged_address #tagged_addr[0]

            paddr = ""
            if 'AddressNumber' in y.keys():
                paddr = (y['AddressNumber'])

            if 'StreetNamePreType' in y.keys():
                paddr = " ".join([paddr, y['StreetNamePreType']])

            if 'StreetNamePreDirectional' in y.keys():
                paddr = " ".join([paddr, y["StreetNamePreDirectional"]])

            if 'StreetName' in y.keys():
                paddr = " ".join([paddr, y["StreetName"]])

            if 'StreetNamePostDirectional' in y.keys():
                paddr = " ".join([paddr, y["StreetNamePostDirectional"]])

            if 'StreetNamePostType' in y.keys():
                if y["StreetNamePostType"].lower() in usaddress.STREET_NAMES:
                    paddr = " ".join([paddr, y["StreetNamePostType"]])

            if 'OccupancyIdentifier' in y.keys():
                if debug == 1:
                    print("we have an occupancy id")
                if not 'OccupancyType' in y.keys():
                    if debug ==1:
                        print("we do not have an occ type")
                    paddr = " ".join([paddr, y['OccupancyIdentifier']])

            if 'PlaceName' in y.keys():
                paddr = " ".join([paddr, y['PlaceName']])

            if paddr == "":
                paddr = address # set it back to original input
                cleaning_note = "|parsed usaddress.tag returned no string so resetting"

        else:
            paddr = address
            if debug == 1:
                print("Returning pre-cleaned address: " + paddr)
            cleaning_note = cleaning_note + "|returning pre-cleaned address"
    
    except usaddress.RepeatedLabelError as e :
        cleaning_note = cleaning_note + "|usaddress.RepeatedLabelError so returning pre-cleaned address"
        paddr = address
        
    if debug > 0:
        print("Cleaning note: " + cleaning_note)
    
    counter = counter + 1
   
    return paddr, cleaning_note

## Goal - clean up street component of the address
* City,  state, and zip are usually fine
* Any component that is known, eg all addresses are in the state of Calfornia, should manually set that value for all rows, eg:
    * df['state'] = 'CA'

## Read in the data to be cleaned

First identify one or more CSV files to process.

In [ ]:
# Take a look at our data to be processed - here in the indata directory
!ls indata

In [ ]:
#identify file(s) to be processed

## One way to get a list of one or more files
## UNCOMMENT to use this approach
#my_files = !ls indata/*.csv
#my_files

## Or just identify the one file to be processed
my_files = ['indata/sample_addresses2.csv']

### Read in the file or files
Chunk the file into separtate files of size **chunksize** if the data are larger than 100,000. You can increase this chunksize depending on your computer configuration.

In [ ]:
for infile in my_files:

    print("Processing File: ", infile)
    
    chunksize = 100000 # max number of rows to process in case we have millions
    file_counter = 0 # keep track of the number of files we output
    
    # Now read in the file in chunks
    ## Note: you may need to change the column names 
    ## but you should specify the dtype for each
    for df in pd.read_csv(infile, chunksize=chunksize, encoding="latin-1", dtype={"id": str, "address": str, "city": str, 'zip': str, 'state': str}):
        
        file_counter = file_counter+1
        print("\nprocessing chunk: " + str(file_counter))
        
        # Clean the addresses
        df['clean_street'], df['clean_notes'] = zip(*df['street'].map(get_clean_street))
        
        # Sort the addresses - makes geocoding faster
        df.sort_values(by=['city', 'zip'], inplace=True)
        
        # Check for and create output directory if needed
        outdir = os.path.join(os.getcwd(),"cleaned")
        if not os.path.isdir(outdir):
            # If the output directory does not exist create it
            !mkdir {outdir}
            print('\nOutput directory has been created:' + outdir)
        else:
            print("\nOutput directory exists and is: " + outdir)
            
        # Write cleaned addresses to csv file for geocoding
        if infile.find("/") >=0:
            file_prefix = infile.split("/")[1].split(".")[0]
        else:
            file_prefix =i[0:-4]
    
        outfile = outdir + "/"+ file_prefix +"_cleaned_" + str(file_counter) + ".csv"
        df.to_csv(outfile, index=False)
        
        print("Saving cleaned addresses to: " + outfile)

In [ ]:
# Check ouput
!ls {outdir}

### About the output

The **get_clean_street** function appends two columns to the output dataframe:

- `clean_street` which has the cleaned address (single line format) 
- `clean_notes` any notes generated by the cleaning function.

The only address data that is cleaned is the data contained in the column mapped to the function, e.g. **street** in the example shown below:

> df['clean_street'], df['clean_notes'] = zip(*df['street'].map(get_clean_street))


*Be sure to set the correct column to be cleaned - needs to match column label in your dataframe!*

### DONE

---
Created by Patty Frontiera, UC Berkeley
Last updated Nov 11, 2019